In [244]:
%reset -f
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['svg.fonttype'] = 'none'
mpl.rcParams['figure.dpi']= 300

# input='/home/isacco.cenacchi/data/Tirocinio/out/tools_comparison.tsv'
# input='/Users/isaccocenacchi/Desktop/Tirocinio/out/MAGs_mini_CRISPRtools/MAGs_mini_tools_comparison.tsv'
input='/Users/isaccocenacchi/Desktop/Tirocinio/out/MAGs_short_tools_comparison.tsv'


df=pd.read_csv(input, sep='\t', index_col=False, header=0)


# Calcolo delle lunghezze delle spacers e dei repeat
df['SP_lens'] = df['Spacers'].apply(lambda x: [len(spacer) for spacer in x.split(',')])
df['DR_lens'] = df['Repeats'].apply(lambda x: [len(dr) for dr in x.split(',')])
# Calcolo delle mediane
df['median_DR_len'] = df['DR_lens'].apply(lambda x: pd.Series(x).median())
df['median_SP_len'] = df['SP_lens'].apply(lambda x: pd.Series(x).median())
# Calcolo max e min DR e SP
df['max_DR_len'] = df['DR_lens'].apply(lambda x: pd.Series(x).max())
df['min_DR_len'] = df['DR_lens'].apply(lambda x: pd.Series(x).min())
df['max_SP_len'] = df['SP_lens'].apply(lambda x: pd.Series(x).max())
df['min_SP_len'] = df['SP_lens'].apply(lambda x: pd.Series(x).min())

# Funzione per calcolare il coefficiente di variazione
def coeff_var(x):
    return pd.Series(x).std() / pd.Series(x).mean()

# Deviazione standard
df['std_DR_len'] = df['DR_lens'].apply(lambda x: pd.Series(x).std())
df['std_SP_len'] = df['SP_lens'].apply(lambda x: pd.Series(x).std())

# Coefficiente di variazione
df['cv_DR_len'] = df['DR_lens'].apply(coeff_var)
df['cv_SP_len'] = df['SP_lens'].apply(coeff_var)

# Rapporto tra la lunghezza minima e massima
df['minmax_DR_ratio'] = df['DR_lens'].apply(lambda x: min(x) / max(x) if max(x) != 0 else 0)

# Separare i tool, esplodendo le righe in base ai tool multipli
df_exploded = df.assign(ToolCodename=df['ToolCodename'].str.split(',')).explode('ToolCodename')
# Rimuovere eventuali spazi bianchi dai nomi dei tool
df_exploded['ToolCodename'] = df_exploded['ToolCodename'].str.strip()

# ordino le righe ib base al nome del tool
df_exploded = df_exploded.sort_values(by='ToolCodename')

# Lista dei tool e delle statistiche
tools = list(df_exploded['ToolCodename'].unique())
stats={'median_DR_len':'Lunghezza Mediana DR',
       'median_SP_len':'Lunghezza Mediana SP',
       'min_DR_len':'Lunghezza Minima DR',
       'max_DR_len':'Lunghezza Massima DR',
       'min_SP_len':'Lunghezza Minima SP',
       'max_SP_len':'Lunghezza Massima SP',
       'std_DR_len':'Deviazione Standard della Lunghezza DR',
       'std_SP_len':'Deviazione Standard della Lunghezza SP',
       'cv_DR_len':'Coeffiente di Variazione della Lunghezza DR',
       'cv_SP_len':'Coeffiente di Variazione della Lunghezza SP',
       'minmax_DR_ratio':'Rapporto lunghezza minima/massima DR'}

In [ ]:
# N. CRISPR per tool

# dict_tool_count = df_exploded['ToolCodename'].value_counts().to_dict()
df_exploded['ToolCodename'].value_counts().plot(kind='bar', title='CRISPR count per tool')
# scrivo il numero di CRISPR sopra le barre
for i, v in enumerate(df_exploded['ToolCodename'].value_counts()):
    plt.text(i - 0.3, v + 3, str(v))
plt.show()

# N. di Cluster ID univoci per tool
df_exploded.groupby('ToolCodename')['ID'].nunique().sort_values(ascending=False).plot(kind='bar', title='Unique ID count per tool')
# # scrivo il numero di ID univoci sopra le barre
for i, v in enumerate(df_exploded.groupby('ToolCodename')['ID'].nunique().sort_values(ascending=False)):
    plt.text(i - 0.3, v + 3, str(v))
plt.show()




In [ ]:
# UPSET PLOT
from upsetplot import UpSet,  generate_counts, from_memberships, plot

df_upset = from_memberships(df.ToolCodename.str.split(","), data=df)

# Genera un UpSet plot con i conteggi
upset=UpSet(df_upset, subset_size='count', show_counts=True, min_subset_size=1, intersection_plot_elements=1)

# upset.style_subsets(present=["minced_Paper", "minced_Default"], max_degree=2, min_degree=1, facecolor="blue")
# upset.style_subsets(present="minced_Paper", facecolor="blue")
upset.add_catplot(value="max_DR_len", kind="strip", color="red")

upset_plot = upset.plot()
upset_plot["intersections"].set_ylabel("Subset size")
upset_plot["totals"].set_xlabel("Nº of CRISPRs")
plt.suptitle("Tool comparison")
plt.show()

In [ ]:
# UPSET PLOT con tool selezionati
from upsetplot import UpSet,  generate_counts, from_indicators

# Crea un dataframe con colonne per ogni tool e True/False
tool_indicator = pd.DataFrame(
    [{cat: True for cat in cats} for cats in df.ToolCodename.str.split(",").values]
).fillna(False)


# Aggiunge le colonne degli indicatori al dataframe originale
df_upset = pd.concat([df, tool_indicator], axis=1)

# Crea un dataframe per l'UpSet che ha come indice i valori delle colonne selezionate
df_upset = from_indicators(
    ["CRISPRDetect3", "CRISPRDetect3_online", "CRISPRDetect3_cpu", "CRISPRDetect3_nocpu", "CRISPRDetect2", "CRISPRDetect2_cpu", "CRISPRDetect2_nocpu"],
    data=df_upset
)

# Genera un UpSet plot con i conteggi
upset = UpSet(df_upset, subset_size='count', show_counts=True, max_subset_size=500, intersection_plot_elements=1)
upset.add_catplot(value="max_DR_len", kind="strip", color="red")
upset_plot = upset.plot()
upset_plot["intersections"].set_ylabel("Subset size")
upset_plot["totals"].set_xlabel("Nº of CRISPRs")
plt.suptitle("Tool comparison")
plt.show()

In [ ]:
# SUPERVENN PLOT
from supervenn import supervenn, make_sets_from_chunk_sizes

# Crea un dataframe con colonne per ogni tool e True/False
tool_indicator = pd.DataFrame(
    [{cat: True for cat in cats} for cats in df.ToolCodename.str.split(",").values]
).fillna(False)

# Creo un dataframe per l'indicatore supervenn
supervenn_indicator = tool_indicator.groupby(list(tool_indicator.columns)).size().reset_index(name='size')

sets, labels = make_sets_from_chunk_sizes(supervenn_indicator)

plt.figure(figsize=(16, 8))
supervenn(sets, labels, min_width_for_annotation=5)


In [ ]:
# Diagramma di venn con pyvenn (2-6), 
from venn import venn

# Seleziona i tool che ti interessano
tools_of_interest = ['pilercr_PILER1', 'minced_Paper', 'CRISPRDetect2', 'CRISPRDetect3']

# Crea un dizionario di set dove le chiavi sono i tool e i valori sono i set di ID per ogni tool
tool_sets_id = {}
for tool in tools_of_interest:
    tool_sets_id[tool] = set(df_exploded[df_exploded['ToolCodename'] == tool]['ID'])

# Crea un dizionario di set dove le chiavi sono i tool e i valori sono i set di index per ogni tool
tool_sets_index = {}
for tool in tools_of_interest:
    tool_sets_index[tool] = set(df_exploded[df_exploded['ToolCodename'] == tool].index)

# # Itera su ogni chiave e calcola la dimensione del set
# for key, value in tool_sets_index.items():
#     print(f"La chiave '{key}' ha {len(value)} elementi.")
# print('\n')
# # Itera su ogni chiave e calcola la dimensione del set
# for key, value in tool_sets_id.items():
#     print(f"La chiave '{key}' ha {len(value)} elementi.")

# Crea il grafico di Venn
venn(tool_sets_id)
plt.title('ID Venn diagram')
plt.show()

venn(tool_sets_index)
plt.title('Index Venn diagram')
plt.show()

In [ ]:
# Statistiche per Tool

tools = list(df_exploded['ToolCodename'].unique())

stats={'median_DR_len':'Lunghezza Mediana DR',
       'median_SP_len':'Lunghezza Mediana SP',
       'min_DR_len':'Lunghezza Minima DR',
       'max_DR_len':'Lunghezza Massima DR',
       'min_SP_len':'Lunghezza Minima SP',
       'max_SP_len':'Lunghezza Massima SP',
       'std_DR_len':'Deviazione Standard della Lunghezza DR',
       'std_SP_len':'Deviazione Standard della Lunghezza SP',
       'cv_DR_len':'Coeffiente di Variazione della Lunghezza DR',
       'cv_SP_len':'Coeffiente di Variazione della Lunghezza SP',
       'minmax_DR_ratio':'Rapporto lunghezza minima/massima DR'}

for tool in tools:
    for stat in stats:
        sns.histplot(df_exploded[df_exploded['ToolCodename'] == tool][stat], kde=True)
        plt.title(f'{stats[stat]} per {tool}')
        plt.xlabel(stats[stat])
        plt.yscale('log') # Con scala logaritmica
        plt.show()

In [ ]:
# Statistiche globali

# Visualizzazione delle mediane DR
sns.histplot(df['median_DR_len'], kde=True)
plt.title('Distribuzione della lunghezza mediana DR')
plt.xlabel('Lunghezza Mediana DR')
plt.yscale('log')
plt.show()

# Visualizzazione delle mediane SP
sns.histplot(df['median_SP_len'], kde=True)
plt.title('Distribuzione della lunghezza mediana SP')
plt.xlabel('Lunghezza Mediana SP')
plt.show()

# Visualizzazione dei min DR
sns.histplot(df['min_DR_len'], kde=True)
plt.title('Distribuzione della lunghezza minima DR')
plt.xlabel('Lunghezza Minima DR')
plt.show()

# Visualizzazione dei max DR
sns.histplot(df['max_DR_len'], kde=True)
plt.title('Distribuzione della lunghezza massima DR')
plt.xlabel('Lunghezza Massima DR')
plt.show()

# Visualizzazione dei min SP
sns.histplot(df['min_SP_len'], kde=True)
plt.title('Distribuzione della lunghezza minima SP')
plt.xlabel('Lunghezza Minima SP')
plt.show()

# Visualizzazione dei max SP
sns.histplot(df['max_SP_len'], kde=True)
plt.title('Distribuzione della lunghezza massima SP')
plt.xlabel('Lunghezza Massima SP')
plt.show()

# Visualizzazione della std DR
sns.histplot(df['std_DR_len'], kde=True)
plt.title('Distribuzione della deviazione standard DR')
plt.xlabel('Deviazione Standard DR')
plt.show()

# Visualizzazione della std SP
sns.histplot(df['std_SP_len'], kde=True)
plt.title('Distribuzione della deviazione standard SP')
plt.xlabel('Deviazione Standard SP')
plt.show()

# Visualizzazione del coefficiente di variazione DR
sns.histplot(df['cv_DR_len'], kde=True)
plt.title('Distribuzione del coefficiente di variazione DR')
plt.xlabel('Coefficient of Variation DR')
plt.show()

# Visualizzazione del coefficiente di variazione SP
sns.histplot(df['cv_SP_len'], kde=True)
plt.title('Distribuzione del coefficiente di variazione SP')
plt.xlabel('Coefficient of Variation SP')
plt.show()

# Visualizzazione del rapporto tra lunghezza minima e massima DR
sns.histplot(df['minmax_DR_ratio'], kde=True, log_scale=True)
plt.title('Distribuzione del rapporto tra lunghezza minima e massima DR')
plt.xlabel('Rapporto lunghezza minima/massima DR')
plt.show()


In [ ]:
# Statistiche globali con scala logaritmica

# Visualizzazione della lunghezza mediana DR con scala logaritmica
ax = df['median_DR_len'].plot(kind='hist', bins=50)
ax.set_title('Distribuzione della lunghezza mediana DR')
ax.set_yscale('log')
plt.show()

# Visualizzazione della lunghezza mediana SP con scala logaritmica
ax = df['median_SP_len'].plot(kind='hist', bins=50)
ax.set_title('Distribuzione della lunghezza mediana SP')
ax.set_yscale('log')
plt.show()

# Visualizzazione della lunghezza minima DR con scala logaritmica
ax = df['min_DR_len'].plot(kind='hist', bins=50)
ax.set_title('Distribuzione della lunghezza minima DR')
ax.set_yscale('log')
plt.show()

# Visualizzazione della lunghezza massima DR con scala logaritmica
ax = df['max_DR_len'].plot(kind='hist', bins=50)
ax.set_title('Distribuzione della lunghezza massima DR')
ax.set_yscale('log')
plt.show()

# Visualizzazione della lunghezza minima SP con scala logaritmica
ax = df['min_SP_len'].plot(kind='hist', bins=50)
ax.set_title('Distribuzione della lunghezza minima SP')
ax.set_yscale('log')
plt.show()

# Visualizzazione della lunghezza massima SP con scala logaritmica
ax = df['max_SP_len'].plot(kind='hist', bins=50)
ax.set_title('Distribuzione della lunghezza massima SP')
ax.set_yscale('log')
plt.show()

# Visualizzazione della deviazione standard DR con scala logaritmica
ax = df['std_DR_len'].plot(kind='hist', bins=50)
ax.set_title('Distribuzione della deviazione standard DR')
ax.set_yscale('log')
plt.show()

# Visualizzazione della deviazione standard SP con scala logaritmica
ax = df['std_SP_len'].plot(kind='hist', bins=50)
ax.set_title('Distribuzione della deviazione standard SP')
ax.set_yscale('log')
plt.show()

# Visualizzazione del coefficiente di variazione DR con scala logaritmica
ax = df['cv_DR_len'].plot(kind='hist', bins=50)
ax.set_title('Distribuzione del coefficiente di variazione DR')
ax.set_yscale('log')
plt.show()

# Visualizzazione del coefficiente di variazione SP con scala logaritmica
ax = df['cv_SP_len'].plot(kind='hist', bins=50)
ax.set_title('Distribuzione del coefficiente di variazione SP')
ax.set_yscale('log')
plt.show()

# Visualizzazione del rapporto tra lunghezza minima e massima DR con scala logaritmica
ax = df['minmax_DR_ratio'].plot(kind='hist', bins=50)
ax.set_title('Distribuzione del rapporto tra lunghezza minima e massima DR')
ax.set_yscale('log')
plt.show()

In [ ]:
# Visualizzione dei dati divisa per ToolCodename con Boxplot

y = list(stats.items())[0]
# median_DR_len     0
# median_SP_len     1
# min_DR_len        2
# max_DR_len        3
# min_SP_len        4
# max_SP_len        5
# std_DR_len        6
# std_SP_len        7
# cv_DR_len         8
# cv_SP_len         9
# minmax_DR_ratio   10


# Applica un tema estetico di Seaborn
# sns.set_theme(style="whitegrid", palette="muted")

# Crea la figura e l'asse
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))  # Ridotto l'altezza per renderlo meno schiacciato

# Boxplot per visualizzare la distribuzione della lunghezza mediana dei DR
sns.boxplot(data=df_exploded, x='ToolCodename', y=y[0], color='white', 
            flierprops={'alpha':0}, medianprops={'color':'k'}, linewidth=1.5, width=0.6)

# Stripplot per visualizzare i singoli punti, con un colore più brillante
sns.stripplot(data=df_exploded, ax=ax, x='ToolCodename', y=y[0], palette='Set2', 
              alpha=0.7, jitter=0.25, size=5, hue='ToolCodename', legend=False)

# Migliora l'estetica generale
sns.despine()

# Personalizzazione delle etichette
ax.set_xlabel('Tool', size=13, labelpad=15)
ax.set_ylabel(y[1], size=13, labelpad=10)
ax.set_title(y[1], size=14, weight='bold')

# Ruota le etichette per una migliore leggibilità
plt.xticks(rotation=45, ha='right', size=11)  # Rotazione a 45 gradi con allineamento a destra

# Personalizza le etichette sull'asse Y
for y in ax.get_yticklabels():
    y.set_size(12)
# aumenta i tick sull'asse y
ax.yaxis.set_major_locator(plt.MaxNLocator(15))
# Migliora lo spessore delle spine e dei tick
ax.spines['bottom'].set_linewidth(1.5)
ax.spines['left'].set_linewidth(1.5)
ax.tick_params(width=1.5, length=6)  # Tick più lunghi

# Opzionalmente, puoi settare i limiti dell'asse Y per focalizzare i dati (decommenta se serve)
# ax.set_ylim(0, 1)

# Mostra il grafico
plt.tight_layout()  # Per evitare sovrapposizioni
plt.show()


In [ ]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

seq1 = "AGCTAGC"
seq2 = "AGCTGAC"

alignments = pairwise2.align.globalxx(seq1, seq2)
for alignment in alignments:
    print(format_alignment(*alignment))

In [ ]:
from Bio.Align import PairwiseAligner
import numpy as np

# Inizializza l'allineatore
aligner = PairwiseAligner()

sequences = ["AGCTAGC", "AGCTGAC", "AGCTA", "AGCTAGG"]

# Inizializza la matrice di similarità
similarity_matrix = np.zeros((len(sequences), len(sequences)))

for i in range(len(sequences)):
    for j in range(i, len(sequences)):
        # Calcola il punteggio di allineamento
        score = aligner.score(sequences[i], sequences[j])
        similarity_matrix[i][j] = score
        similarity_matrix[j][i] = score  # La matrice è simmetrica

print(similarity_matrix)

In [ ]:
from collections import Counter

sequences = ["AGCTAGC", "AGCTGAC", "AGCTA", "AGCTAGC"]
counter = Counter(sequences)
most_common_sequence, count = counter.most_common(1)[0]

print(f"La sequenza più comune è: {most_common_sequence} con {count} occorrenze.")


In [ ]:
import Levenshtein as lev

# Funzione per calcolare la similarità media tra le ripetizioni
def conservation(dr_list):
    total_similarity = 0
    comparisons = 0
    for i in range(len(dr_list)):
        for j in range(i + 1, len(dr_list)):
            similarity = 1 - (lev.distance(dr_list[i], dr_list[j]) / max(len(dr_list[i]), len(dr_list[j])))
            total_similarity += similarity
            comparisons += 1
    return total_similarity / comparisons if comparisons > 0 else 0

# Calcoliamo la conservazione per ciascun insieme di DR
df['DR_conservation'] = df['Repeats'].apply(lambda x: conservation(x.split(',')))
